In [1]:
import pandas as pd,numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,f1_score,confusion_matrix
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_data = pd.read_csv(r'C:\Users\sadiq\Documents\Employment Test\train.csv')
test_data = pd.read_csv(r'C:\Users\sadiq\Documents\Employment Test\test.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
train_data.head()

,index,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,81041,8.610000e+11,Y,"Martinez, Duffy and Bird",3.375,272000,180,01-01-2012,Mar-12,72,2,31,770,B12,0,786,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,51021,1.730000e+11,Y,"Swanson, Newton and Miller",4.250,371000,360,01-01-2012,Mar-12,95,1,45,727,C86,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,83866,4.820000e+11,Y,OTHER,4.500,90000,360,01-01-2012,Mar-12,69,1,39,726,B12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,92311,6.770000e+11,Y,"Turner, Baldwin and Rhodes",4.500,47000,240,01-02-2012,Apr-12,75,1,43,688,B12,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,72274,4.330000e+11,X,OTHER,4.875,177000,360,01-01-2012,Mar-12,80,2,13,675,C86,0,672,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
tr_df = train_data.iloc[:,2:]
tr_df = tr_df.drop(['origination_date','first_payment_date'],1)
tr_df.head()



,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,Y,"Martinez, Duffy and Bird",3.375,272000,180,72,2,31,770,B12,0,786,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Y,"Swanson, Newton and Miller",4.250,371000,360,95,1,45,727,C86,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Y,OTHER,4.500,90000,360,69,1,39,726,B12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Y,"Turner, Baldwin and Rhodes",4.500,47000,240,75,1,43,688,B12,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,X,OTHER,4.875,177000,360,80,2,13,675,C86,0,672,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
dataframe_con = tr_df.select_dtypes(include=np.number)
dataframe_cat = tr_df.select_dtypes(exclude=np.number)

In [7]:
le = LabelEncoder()
for col in dataframe_cat:
    tr_df[col] = le.fit_transform(tr_df[col])
     

tr_df.head()

,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,1,5,3.375,272000,180,72,2,31,770,1,0,786,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,15,4.250,371000,360,95,1,45,727,2,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,8,4.500,90000,360,69,1,39,726,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,18,4.500,47000,240,75,1,43,688,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,8,4.875,177000,360,80,2,13,675,2,0,672,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler
X = tr_df.drop('m13',1)
y = tr_df['m13']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state = 92)
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

lr = LogisticRegression(random_state=11)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print("F1 Score : ",f1_score(y_test, y_pred))
print("Confusion Matrix: \n",confusion_matrix(y_test,y_pred))
print("=="*20)
print("Classification Report: \n",classification_report(y_test,y_pred))

F1 Score :  0.24786324786324787
Confusion Matrix: 
 [[36934     3]
 [  173    29]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     36937
           1       0.91      0.14      0.25       202

    accuracy                           1.00     37139
   macro avg       0.95      0.57      0.62     37139
weighted avg       0.99      1.00      0.99     37139



In [9]:
from imblearn.over_sampling import RandomOverSampler
sample = RandomOverSampler(random_state=11)
# fit_sample on training data
X_sample, y_sample = sample.fit_sample(X_train, y_train)
lr = LogisticRegression(random_state=11)
lr.fit(X_sample, y_sample)
y_pred = lr.predict(X_test)
print(f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.07760141093474428
              precision    recall  f1-score   support

           0       1.00      0.90      0.95     36937
           1       0.04      0.76      0.08       202

    accuracy                           0.90     37139
   macro avg       0.52      0.83      0.51     37139
weighted avg       0.99      0.90      0.94     37139



In [10]:
y_train = np.array(y_train).reshape((-1, ))
y_test = np.array(y_test).reshape((-1, ))

lr = LogisticRegression(random_state=11)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
y_pred_new = np.array(y_pred).reshape((-1, ))
print("F1 Score : ",f1_score(y_test, y_pred_new))
print("Confusion Matrix: \n",confusion_matrix(y_test,y_pred))
print("=="*20)
print("Classification Report: \n",classification_report(y_test,y_pred))

F1 Score :  0.24786324786324787
Confusion Matrix: 
 [[36934     3]
 [  173    29]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     36937
           1       0.91      0.14      0.25       202

    accuracy                           1.00     37139
   macro avg       0.95      0.57      0.62     37139
weighted avg       0.99      1.00      0.99     37139



In [11]:
# test data
ts_df = test_data.iloc[:,2:]
ts_df = ts_df.drop(['origination_date','first_payment_date'],1)

In [12]:
test_con = ts_df.select_dtypes(include=np.number)
test_cat = ts_df.select_dtypes(exclude=np.number)

le = LabelEncoder()
for col in test_cat:
    ts_df[col] = le.fit_transform(ts_df[col])
     

ts_df.head()

,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,0,8,3.375,144000,180,73,2,20,790,0,0,797,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,4.375,110000,360,88,1,45,779,2,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,4.125,243000,240,70,2,40,727,1,0,705,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,8,2.875,189000,120,64,2,35,798,0,0,793,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,8,4.000,141000,360,84,2,31,819,0,12,799,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("F1 Score : ",f1_score(y_test, y_pred))
print("Confusion Matrix: \n",confusion_matrix(y_test,y_pred))
print("=="*20)
print("Classification Report: \n",classification_report(y_test,y_pred))

F1 Score :  0.31058823529411766
Confusion Matrix: 
 [[36780   157]
 [  136    66]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     36937
           1       0.30      0.33      0.31       202

    accuracy                           0.99     37139
   macro avg       0.65      0.66      0.65     37139
weighted avg       0.99      0.99      0.99     37139



In [14]:
from imblearn.under_sampling import TomekLinks

# # Instantiate smote
sample = TomekLinks()

# # fit_sample on training data
X_train_tmksample, y_train_tmksample = sample.fit_sample(X_train, y_train)
lr.fit(X_train_tmksample, y_train_tmksample)
y_pred1 = lr.predict(X_test)
print(f1_score(y_test, y_pred1))

0.270042194092827


In [15]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train_tmksample, y_train_tmksample)
y_pred = clf.predict(X_test)
print("F1 Score : ",f1_score(y_test, y_pred))
print("Confusion Matrix: \n",confusion_matrix(y_test,y_pred))
print("=="*20)
print("Classification Report: \n",classification_report(y_test,y_pred))

F1 Score :  0.32054176072234764
Confusion Matrix: 
 [[36767   170]
 [  131    71]]
Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     36937
           1       0.29      0.35      0.32       202

    accuracy                           0.99     37139
   macro avg       0.65      0.67      0.66     37139
weighted avg       0.99      0.99      0.99     37139



In [16]:
from sklearn.ensemble import RandomForestClassifier
rf_sample = RandomForestClassifier(random_state=11)
# # fit model on training data
#rf_sample.fit(X_sample1, y_sample1)
rf_sample.fit(X_train_tmksample, y_train_tmksample)
# #predicting on test data
#y_pred_sample = rf_sample.predict(X_test)
y_pred_sample = rf_sample.predict(X_test)
print("F1 Score : ",f1_score(y_test, y_pred_sample))

F1 Score :  0.5100671140939597


In [17]:
from sklearn.naive_bayes import MultinomialNB

#Naive Bayes Model
nb = MultinomialNB()

nb.fit(X_train_tmksample, y_train_tmksample)

y_pred = nb.predict(X_test)

print("F1 Score : ",f1_score(y_test, y_pred))


F1 Score :  0.28340080971659914


In [18]:
from sklearn.svm import LinearSVC

#Linear SVM Model
lsvm = LinearSVC(random_state=1992)

lsvm.fit(X_train_tmksample, y_train_tmksample)

y_pred = lsvm.predict(X_test)

print("F1 Score : ",f1_score(y_test, y_pred))


F1 Score :  0.37354085603112835


In [19]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(base_estimator = lr, random_state = 1992)
xgb_clf.fit(X_train_tmksample, y_train_tmksample)
y_pred = xgb_clf.predict(X_test)
print("F1 Score : ",f1_score(y_test, y_pred))

F1 Score :  0.5142857142857143


In [21]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train_tmksample, y_train_tmksample)
y_pred_sample = rf_sample.predict(X_test)
print("F1 Score : ",f1_score(y_test, y_pred_sample))

F1 Score :  0.5100671140939597


In [30]:
test_arr = np.array(ts_df)

In [31]:
xgb_clf_pred = xgb_clf.predict(test_arr)
test_result = pd.DataFrame(test_data['loan_id'])
submission = pd.DataFrame({"index": test_data["index"],"m13": xgb_clf_pred})
submission.to_csv((r'C:\Users\sadiq\Documents\Employment Test\loan_submission.csv'),index=False)

In [23]:
ts_df.head()

,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,0,8,3.375,144000,180,73,2,20,790,0,0,797,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,4.375,110000,360,88,1,45,779,2,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,4.125,243000,240,70,2,40,727,1,0,705,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,8,2.875,189000,120,64,2,35,798,0,0,793,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,8,4.000,141000,360,84,2,31,819,0,12,799,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
from sklearn.ensemble import BaggingClassifier

# # Code starts here
bagging_clf = BaggingClassifier(base_estimator = LogisticRegression(), n_estimators = 100, max_samples = 100, random_state=42)

bagging_clf.fit(X_train_tmksample, y_train_tmksample)

y_pred_sample = bagging_clf.predict(X_test)

print(f1_score(y_test, y_pred_sample))

0.0


In [35]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result =  grid_search.fit(X_train_tmksample, y_train_tmksample)
y_pred_sample = grid_search.predict(X_test)

print(f1_score(y_test, y_pred_sample))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.54421768707483
Best: 0.995914 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.995914 (0.000515) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.995884 (0.000498) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.995914 (0.000515) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.995758 (0.000495) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.995752 (0.000483) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.995752 (0.000501) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.995243 (0.000442) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.995243 (0.000442) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.995243 (0.000444) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.994674 (0.000170) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.994674 (0.000170) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.994680 (0.000165) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.9945

In [39]:
solvers = ['newton-cg']
penalty = ['l2']
c_values = [100]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result =  grid_search.fit(X_train_tmksample, y_train_tmksample)
y_pred_sample = grid_search.predict(X_test)

print(f1_score(y_test, y_pred_sample))

0.54421768707483
